In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'

In [10]:
response = requests.get(url)

In [11]:
soup = BeautifulSoup (response.content)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse

amazon_search_links = []

def generate_amazon_search_link(product_name):
  base_url = f"https://www.amazon.com/s?k={product_name}&crid=TOQJCXSNICLI&sprefix={product_name}%2Caps%2C163&ref=nb_sb_noss_1"
  search_query = urllib.parse.quote(product_name)
  url = base_url + "k=" + search_query
  return url

# for product_name in good_reads_titles_url_ready:
#   amazon_search_link = generate_amazon_search_link(product_name)
#   amazon_search_links.append(amazon_search_link)

description = []
genres = []
titles = []
authors = []
ratings = []
amazon_search_links = []

with open('description.csv', 'a') as the_file:
    the_file.write(f'page_number;title;author;rating;description;genres;url\n')

for page_number in range(1, 7):
    attemps = 0
    trying = True
    while trying:
        try:
            url_page_numbers = f'https://www.goodreads.com/list/show/1.Best_Books_Ever?page={page_number}'
            response_page_numbers = requests.get(url_page_numbers)
            soup_page_numbers = BeautifulSoup (response_page_numbers.content)
            components = soup_page_numbers.find_all('td', width='100%')
            print(page_number)
            # with open('description.txt', 'a') as the_file:
            #     the_file.write(f'PAGINA NUMERO = {page_number}\n')
            for component in components:
                book_url = component.find('a', class_='bookTitle').get('href')
                # ?page=1
                title = component.find_all('a', class_='bookTitle')
                text_title = ""
                for title in title:
                    text_title = title.text.strip().replace(";", "")
                    titles.append(text_title)
                author = component.find_all('a', class_='authorName')
                text_author = ""
                for author in author:
                    text_author = author.text.strip().replace(";", "")
                    authors.append(text_author)
                rating = component.find_all(class_='minirating')
                text_rating = ""
                for rating in rating:
                    text_rating = ''.join(rating.text.split()[0]).replace(";", "")
                    ratings.append(text_rating)
                book_response = requests.get(f'https://www.goodreads.com{book_url}')
                book_soup = BeautifulSoup(book_response.content)
                book_components = book_soup.find_all('a', class_='Button Button--tag-inline Button--small')
                book_genres = []
                counter = 0
                text_genre = ""
                for component in book_components:
                    text_genre = component.find('span', class_='Button__labelItem').text.replace(";", "")
                    book_genres.append(text_genre)
                    counter += 1
                    if counter == 7:
                        break
                genres.append(book_genres)
                csv_genres = str(book_genres).replace("[", "").replace("]", "").replace(", ", ",") \
                .replace("'", "").replace(";", "")

                # book_response = requests.get(f'https://www.goodreads.com{book_url}')
                # book_soup = BeautifulSoup(book_response.content)
                description_components = book_soup.find_all('div', class_='BookPageMetadataSection__description')
                book_description = []
                text_description = ""
                for component in description_components:
                    text_description = component.find('span', class_='Formatted').text.replace(";", "").replace("\n", " ")
                    book_description.append(text_description)
                description.append(book_description)

                title_for_url = text_title.split("(")[0].replace(" ", "") if "(" in text_title else text_title.replace(" ", "")
                amazon_search_link = generate_amazon_search_link(title_for_url).replace(";", "")
                amazon_search_links.append(amazon_search_link)

                with open('description.csv', 'a') as the_file:
                    the_file.write(f'{page_number};{text_title};{text_author};{text_rating};{text_description};{csv_genres};{amazon_search_link}\n')
                # with open('description.txt', 'a') as the_file:
                #     the_file.write(f'{text_title}  - {book_genres}\n')   
                trying = False
        except requests.exceptions.Timeout:
            print(" Time out error")
            print(attemps)
            time.sleep(3)
            attemps += 1
            if attemps > 2:
                trying = False

In [70]:
books_df = pd.read_csv("description.csv", delimiter=';')
books_df

,page_number,title,author,rating,description,genres,url
0,1,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.33,NaN,NaN,https://www.amazon.com/s?k=TheHungerGames&crid...
1,1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50,Harry Potter is about to start his fifth year ...,"Young Adult,Fiction,Magic,Childrens,Audiobook,...",https://www.amazon.com/s?k=HarryPotterandtheOr...
2,1,Pride and Prejudice,Jane Austen,4.28,NaN,NaN,https://www.amazon.com/s?k=PrideandPrejudice&c...
3,1,To Kill a Mockingbird,Harper Lee,4.26,The unforgettable novel of a childhood in a sl...,"Classics,Fiction,Historical Fiction,School,Lit...",https://www.amazon.com/s?k=ToKillaMockingbird&...
4,1,The Book Thief,Markus Zusak,4.39,NaN,NaN,https://www.amazon.com/s?k=TheBookThief&crid=T...
...,...,...,...,...,...,...,...
585,6,"Little House in the Big Woods (Little House, #1)",Laura Ingalls Wilder,4.20,NaN,NaN,https://www.amazon.com/s?k=LittleHouseintheBig...
586,6,Macbeth,William Shakespeare,3.90,"One night on the heath, the brave and respecte...","Classics,Plays,Fiction,School,Drama,Read For S...",https://www.amazon.com/s?k=Macbeth&crid=TOQJCX...
587,6,I'll Give You the Sun,Jandy Nelson,4.16,"At first, Jude and her twin brother Noah, are ...","Young Adult,Contemporary,Romance,LGBT,Fiction,...",https://www.amazon.com/s?k=I'llGiveYoutheSun&c...
588,6,Lady Chatterley's Lover,D.H. Lawrence,3.50,Lawrence's frank portrayal of an extramarital ...,"Classics,Fiction,Romance,Literature,Banned Boo...",https://www.amazon.com/s?k=LadyChatterley'sLov...


In [71]:
books_df.drop('page_number', axis=1, inplace=True)

In [73]:
books_df.isnull().sum()

title            0
author           0
rating           0
description    202
genres         202
url              0
dtype: int64

In [74]:
books_df = books_df.dropna()

In [75]:
books_df

,title,author,rating,description,genres,url
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50,Harry Potter is about to start his fifth year ...,"Young Adult,Fiction,Magic,Childrens,Audiobook,...",https://www.amazon.com/s?k=HarryPotterandtheOr...
3,To Kill a Mockingbird,Harper Lee,4.26,The unforgettable novel of a childhood in a sl...,"Classics,Fiction,Historical Fiction,School,Lit...",https://www.amazon.com/s?k=ToKillaMockingbird&...
6,Animal Farm,George Orwell,3.98,Librarian's note: There is an Alternate Cover ...,"Classics,Fiction,Dystopia,Fantasy,School,Polit...",https://www.amazon.com/s?k=AnimalFarm&crid=TOQ...
7,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,4.61,"This four-volume, boxed set contains J.R.R. To...","Fantasy,Fiction,Classics,Adventure,Science Fic...",https://www.amazon.com/s?k=J.R.R.Tolkien4-Book...
8,The Chronicles of Narnia (The Chronicles of Na...,C.S. Lewis,4.27,Librarian note: An alternate cover for this ed...,"Fantasy,Classics,Fiction,Young Adult,Childrens...",https://www.amazon.com/s?k=TheChroniclesofNarn...
...,...,...,...,...,...,...
583,Nine Stories,J.D. Salinger,4.18,Nine Stories (1953) is a collection of short s...,"Short Stories,Fiction,Classics,Literature,Amer...",https://www.amazon.com/s?k=NineStories&crid=TO...
584,"Anne of Avonlea (Anne of Green Gables, #2)",L.M. Montgomery,4.22,When twelve-year-old Anne Shirley came to Avon...,"Classics,Fiction,Young Adult,Historical Fictio...",https://www.amazon.com/s?k=AnneofAvonlea&crid=...
586,Macbeth,William Shakespeare,3.90,"One night on the heath, the brave and respecte...","Classics,Plays,Fiction,School,Drama,Read For S...",https://www.amazon.com/s?k=Macbeth&crid=TOQJCX...
587,I'll Give You the Sun,Jandy Nelson,4.16,"At first, Jude and her twin brother Noah, are ...","Young Adult,Contemporary,Romance,LGBT,Fiction,...",https://www.amazon.com/s?k=I'llGiveYoutheSun&c...


In [77]:
books_df['rating'] = pd.to_numeric(books_df['rating'], errors='coerce')

/var/folders/xb/vbkygnls2kq6z12lrq5ymftc0000gn/T/ipykernel_15346/1164975442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df['rating'] = pd.to_numeric(books_df['rating'], errors='coerce')


In [78]:
books_df['rating'] = books_df['rating'].astype(float)

/var/folders/xb/vbkygnls2kq6z12lrq5ymftc0000gn/T/ipykernel_15346/4024314579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df['rating'] = books_df['rating'].astype(float)


In [79]:
books_df['rating'] = books_df['rating'].apply(lambda x: x * 2)

/var/folders/xb/vbkygnls2kq6z12lrq5ymftc0000gn/T/ipykernel_15346/4115907084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df['rating'] = books_df['rating'].apply(lambda x: x * 2)


In [80]:
books_df

,title,author,rating,description,genres,url
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,9.00,Harry Potter is about to start his fifth year ...,"Young Adult,Fiction,Magic,Childrens,Audiobook,...",https://www.amazon.com/s?k=HarryPotterandtheOr...
3,To Kill a Mockingbird,Harper Lee,8.52,The unforgettable novel of a childhood in a sl...,"Classics,Fiction,Historical Fiction,School,Lit...",https://www.amazon.com/s?k=ToKillaMockingbird&...
6,Animal Farm,George Orwell,7.96,Librarian's note: There is an Alternate Cover ...,"Classics,Fiction,Dystopia,Fantasy,School,Polit...",https://www.amazon.com/s?k=AnimalFarm&crid=TOQ...
7,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,9.22,"This four-volume, boxed set contains J.R.R. To...","Fantasy,Fiction,Classics,Adventure,Science Fic...",https://www.amazon.com/s?k=J.R.R.Tolkien4-Book...
8,The Chronicles of Narnia (The Chronicles of Na...,C.S. Lewis,8.54,Librarian note: An alternate cover for this ed...,"Fantasy,Classics,Fiction,Young Adult,Childrens...",https://www.amazon.com/s?k=TheChroniclesofNarn...
...,...,...,...,...,...,...
583,Nine Stories,J.D. Salinger,8.36,Nine Stories (1953) is a collection of short s...,"Short Stories,Fiction,Classics,Literature,Amer...",https://www.amazon.com/s?k=NineStories&crid=TO...
584,"Anne of Avonlea (Anne of Green Gables, #2)",L.M. Montgomery,8.44,When twelve-year-old Anne Shirley came to Avon...,"Classics,Fiction,Young Adult,Historical Fictio...",https://www.amazon.com/s?k=AnneofAvonlea&crid=...
586,Macbeth,William Shakespeare,7.80,"One night on the heath, the brave and respecte...","Classics,Plays,Fiction,School,Drama,Read For S...",https://www.amazon.com/s?k=Macbeth&crid=TOQJCX...
587,I'll Give You the Sun,Jandy Nelson,8.32,"At first, Jude and her twin brother Noah, are ...","Young Adult,Contemporary,Romance,LGBT,Fiction,...",https://www.amazon.com/s?k=I'llGiveYoutheSun&c...


In [81]:
books_df.isna().sum()

title          0
author         0
rating         6
description    0
genres         0
url            0
dtype: int64

In [82]:
books_df.rename(columns = {'description':'overview'}, inplace = True)

/var/folders/xb/vbkygnls2kq6z12lrq5ymftc0000gn/T/ipykernel_15346/1366270075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df.rename(columns = {'description':'overview'}, inplace = True)


In [84]:
books_df.to_csv('books_df_with_description.csv', index=False)